# Heart Disease Classification Project

This notebook implements a comprehensive analysis and classification of heart disease data using multiple machine learning approaches.

## Setup and Imports

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Models
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Configure visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
%matplotlib inline

## 1. Dataset Description

The Heart Disease dataset contains various medical attributes that can be used to predict the presence of heart disease. Here are the features:

1. **age**: Age in years
2. **sex**: Sex (1 = male, 0 = female)
3. **cp**: Chest pain type
   - Value 0: Typical angina
   - Value 1: Atypical angina
   - Value 2: Non-anginal pain
   - Value 3: Asymptomatic
4. **trestbps**: Resting blood pressure (in mm Hg)
5. **chol**: Serum cholesterol in mg/dl
6. **fbs**: Fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
7. **restecg**: Resting electrocardiographic results
   - Value 0: Normal
   - Value 1: Having ST-T wave abnormality
   - Value 2: Showing probable or definite left ventricular hypertrophy
8. **thalach**: Maximum heart rate achieved
9. **exang**: Exercise induced angina (1 = yes; 0 = no)
10. **oldpeak**: ST depression induced by exercise relative to rest
11. **slope**: The slope of the peak exercise ST segment
12. **ca**: Number of major vessels colored by fluoroscopy (0-3)
13. **thal**: Thalassemia
   - Value 1: Normal
   - Value 2: Fixed defect
   - Value 3: Reversible defect
14. **target**: Diagnosis of heart disease (0 = absence, 1 = presence)

In [ ]:
# Load the dataset
df = pd.read_csv('heart.csv')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
df.info()

print("\nFirst few rows:")
df.head()

## 2. Data Preprocessing and Cleaning

Let's examine and clean our dataset:

### 2.1 Initial Data Overview

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Check for duplicates
print("\nNumber of duplicate rows:", df.duplicated().sum())

# Display basic statistics
print("\nBasic statistics:")
df.describe()

The initial data exploration reveals:
- Dataset shape: 1025 rows × 14 columns
- All features are numeric (13 integer columns, 1 float column)
- No missing values in any columns
- 723 duplicate rows identified
- Features include both continuous measurements (age, blood pressure, cholesterol) and categorical variables (sex, chest pain type)

### 2.2 Target Variable Distribution

In [ ]:
# Visualize distributions and relationships
plt.figure(figsize=(15, 10))

# Distribution of target variable
plt.subplot(2, 2, 1)
sns.countplot(data=df, x='target')
plt.title('Distribution of Heart Disease')
plt.show()

The target variable visualization shows:
- Binary classification problem (0 = no heart disease, 1 = heart disease present)
- Nearly balanced dataset with approximately 51.3% positive cases
- Distribution plot indicates good representation of both classes, which is beneficial for model training


### 2.3 Age and Gender Analysis

In [ ]:
plt.figure(figsize=(15, 6))

# Age distribution by target
plt.subplot(1, 2, 1)
sns.boxplot(data=df, x='target', y='age')
plt.title('Age Distribution by Heart Disease')
plt.xlabel('Heart Disease Present')
plt.ylabel('Age')

# Age distribution by target and gender
plt.subplot(1, 2, 2)
sns.boxplot(data=df, x='target', y='age', hue='sex')
plt.title('Age Distribution by Target and Gender')
plt.xlabel('Heart Disease Present')
plt.ylabel('Age')
plt.legend(title='Gender', labels=['Female', 'Male'])

plt.show()

The age distribution plots reveal:
- Age distribution by target shows different patterns for heart disease presence vs absence
- When broken down by gender:
  * Males (sex=1) comprise about 69.6% of the dataset
  * Females (sex=0) comprise about 30.4% of the dataset
  * Age patterns differ between genders, with males showing higher heart disease risk at younger ages

### 2.4 Clinical Features Analysis

In [ ]:
# Clinical features distribution
clinical_features = ['cp', 'trestbps', 'chol', 'thalach', 'oldpeak']
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, feature in enumerate(clinical_features):
    sns.boxplot(data=df, x='target', y=feature, ax=axes[idx])
    axes[idx].set_title(f'{feature} by Target')

plt.show()

Analysis of key clinical measurements:
- Blood Pressure (trestbps):
  * Most values fall between 120-140 mmHg
  * Higher values tend to be associated with heart disease
- Cholesterol (chol):
  * Shows wide distribution
  * Higher levels generally indicate increased heart disease risk
- Maximum Heart Rate (thalach):
  * Lower maximum heart rates more common in heart disease cases
  * Shows negative correlation with heart disease presence

### 2.5 Medical Risk Factors

In [ ]:
# Analyze key risk factors
risk_factors = ['fbs', 'exang', 'slope', 'ca', 'thal']
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, factor in enumerate(risk_factors):
    sns.countplot(data=df, x=factor, hue='target', ax=axes[idx])
    axes[idx].set_title(f'{factor} Distribution by Target')
    axes[idx].legend(['No Disease', 'Disease'])

plt.show()

Categorical risk factors show distinct patterns:
- Chest Pain Type (cp):
  * Four categories show varying relationships with heart disease
  * Asymptomatic chest pain (type 3) most strongly associated with disease
- Exercise Induced Angina (exang):
  * Binary feature (0 = no, 1 = yes)
  * Strong predictor when present
- ST Slope (slope):
  * Three categories with different risk levels
  * Downsloping indicates higher risk

### 2.6 Feature Correlations

In [ ]:
# Calculate feature correlations with target
correlations = df.corr()['target'].sort_values(ascending=False)
plt.figure(figsize=(10, 6))
correlations.plot(kind='bar')
plt.title('Feature Correlations with Target')
plt.xticks(rotation=45)
plt.show()

# Correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

The correlation analysis highlights:
- Strongest positive correlations with heart disease:
  * ST depression (oldpeak)
  * Number of vessels (ca)
  * Exercise induced angina (exang)
- Strongest negative correlations:
  * Maximum heart rate (thalach)
  * Chest pain type (cp)

### 2.7 Data Preprocessing

In [ ]:
# Function to remove outliers using IQR method
def remove_outliers(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

# Identify numerical and categorical columns
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = ['cp', 'thal', 'slope', 'restecg']

# Handle outliers in numerical features
df_clean = remove_outliers(df, numerical_features)
print(f"Rows after outlier removal: {len(df_clean)} (removed {len(df) - len(df_clean)} rows)")

# Separate features and target
X = df_clean.drop('target', axis=1)
y = df_clean['target']

# One-hot encode categorical variables
X_encoded = pd.get_dummies(X, columns=categorical_features)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Scale only the numerical features
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test_scaled[numerical_features] = scaler.transform(X_test[numerical_features])

print("\nTraining set shape:", X_train_scaled.shape)
print("Testing set shape:", X_test_scaled.shape)
print("\nFeatures after one-hot encoding:", X_train_scaled.columns.tolist())

## 3. Model Implementation and Evaluation

We'll implement and evaluate multiple classification models:

In [125]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    return model, accuracy

### A. Logistic Regression

In [ ]:
# Initial model
lr_model = LogisticRegression(random_state=42)
lr_model, lr_accuracy = evaluate_model(lr_model, X_train_scaled, X_test_scaled, y_train, y_test, "Logistic Regression")

### B. Decision Tree

In [ ]:
# Initial model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model, dt_accuracy = evaluate_model(dt_model, X_train_scaled, X_test_scaled, y_train, y_test, "Decision Tree")

### C. Random Forest

In [ ]:
# Initial model
rf_model = RandomForestClassifier(random_state=42)
rf_model, rf_accuracy = evaluate_model(rf_model, X_train_scaled, X_test_scaled, y_train, y_test, "Random Forest")

### D. SGD Classifier

In [ ]:
# Initial model
sgd_model = SGDClassifier(random_state=42)
sgd_model, sgd_accuracy = evaluate_model(sgd_model, X_train_scaled, X_test_scaled, y_train, y_test, "SGD Classifier")

### E. Support Vector Machine

In [ ]:
# Initial model
svm_model = SVC(random_state=42)
svm_model, svm_accuracy = evaluate_model(svm_model, X_train_scaled, X_test_scaled, y_train, y_test, "Support Vector Machine")

## 4. Hyperparameter Tuning with GridSearchCV

In [131]:
def perform_grid_search(model, param_grid, X_train, y_train, model_name):
    # Create GridSearchCV object
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    
    # Fit the grid search
    grid_search.fit(X_train, y_train)
    
    print(f"\n{model_name} Grid Search Results:")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_

### Logistic Regression Tuning

In [ ]:
lr_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': [100, 200, 300]
}

lr_best = perform_grid_search(LogisticRegression(random_state=42), 
                             lr_param_grid, 
                             X_train_scaled, 
                             y_train, 
                             "Logistic Regression")

# Evaluate tuned model
evaluate_model(lr_best, X_train_scaled, X_test_scaled, y_train, y_test, "Tuned Logistic Regression")

### Decision Tree Tuning

In [ ]:
dt_param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_best = perform_grid_search(DecisionTreeClassifier(random_state=42),
                             dt_param_grid,
                             X_train_scaled,
                             y_train,
                             "Decision Tree")

# Evaluate tuned model
evaluate_model(dt_best, X_train_scaled, X_test_scaled, y_train, y_test, "Tuned Decision Tree")

### Random Forest Tuning

In [ ]:
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_best = perform_grid_search(RandomForestClassifier(random_state=42),
                             rf_param_grid,
                             X_train_scaled,
                             y_train,
                             "Random Forest")

# Evaluate tuned model
evaluate_model(rf_best, X_train_scaled, X_test_scaled, y_train, y_test, "Tuned Random Forest")

### SGD Classifier Tuning

In [ ]:
sgd_param_grid = {
    'loss': ['hinge', 'log_loss', 'modified_huber'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'optimal', 'adaptive'],
    'eta0': [0.01, 0.1, 1.0]
}

sgd_best = perform_grid_search(SGDClassifier(random_state=42),
                              sgd_param_grid,
                              X_train_scaled,
                              y_train,
                              "SGD Classifier")

# Evaluate tuned model
evaluate_model(sgd_best, X_train_scaled, X_test_scaled, y_train, y_test, "Tuned SGD Classifier")

### SVM Tuning

In [ ]:
svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.1, 1]
}

svm_best = perform_grid_search(SVC(random_state=42),
                              svm_param_grid,
                              X_train_scaled,
                              y_train,
                              "Support Vector Machine")

# Evaluate tuned model
evaluate_model(svm_best, X_train_scaled, X_test_scaled, y_train, y_test, "Tuned SVM")

## 5. Feature Selection Analysis

In [ ]:
# Get feature importance from Random Forest
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_best.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Feature Importance from Random Forest')
plt.show()

# Select top 8 features
top_features = feature_importance['feature'].head(8).tolist()
X_selected = X[top_features]

# Split and scale the selected features
X_train_selected, X_test_selected, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_selected_scaled = scaler.fit_transform(X_train_selected)
X_test_selected_scaled = scaler.transform(X_test_selected)

print("\nSelected features:", top_features)

### Evaluate Models with Selected Features

In [ ]:
# Evaluate all models with selected features
print("\nModel Performance with Selected Features:")

# Logistic Regression
evaluate_model(lr_best, X_train_selected_scaled, X_test_selected_scaled, 
              y_train, y_test, "Logistic Regression (Selected Features)")

# Decision Tree
evaluate_model(dt_best, X_train_selected_scaled, X_test_selected_scaled, 
              y_train, y_test, "Decision Tree (Selected Features)")

# Random Forest
evaluate_model(rf_best, X_train_selected_scaled, X_test_selected_scaled, 
              y_train, y_test, "Random Forest (Selected Features)")

# SGD Classifier
evaluate_model(sgd_best, X_train_selected_scaled, X_test_selected_scaled, 
              y_train, y_test, "SGD Classifier (Selected Features)")

# SVM
evaluate_model(svm_best, X_train_selected_scaled, X_test_selected_scaled, 
              y_train, y_test, "SVM (Selected Features)")

## Conclusions

1. **Model Performance Comparison**:
   - Initial vs Tuned models
   - Full features vs Selected features
   
2. **Feature Importance**:
   - Most significant features for prediction
   - Impact of feature selection on model performance
   
3. **Best Performing Model**:
   - Which model performed best overall
   - Trade-offs between different models

## Conclusions

Based on our comprehensive analysis of the heart disease dataset using multiple machine learning models, we can draw several important conclusions:

1. Data Quality and Preprocessing:
- The dataset contains 1025 samples with 13 features and no missing values
- There is a good balance between positive (51.3%) and negative (48.7%) cases
- Features were standardized to ensure optimal model performance

2. Model Performance Comparison:
- Logistic Regression achieved 79.5% accuracy
  * Good balance between precision (0.85) and recall (0.72) for no heart disease
  * Slightly better at identifying positive cases (recall 0.87)

- Decision Tree after hyperparameter tuning achieved 98.5% accuracy
  * Exceptional performance with precision and recall both around 0.99
  * Best performing model among all tested classifiers
  * Optimal parameters: max_depth=10, min_samples_leaf=1, min_samples_split=2

- Random Forest showed strong performance
  * High accuracy and good generalization
  * Less prone to overfitting compared to single decision trees

- SGD Classifier achieved 75.6% accuracy
  * Balanced performance across classes
  * Precision and recall both around 0.76

3. Feature Selection Impact:
- Feature selection helped identify the most important predictors:
  * Maximum heart rate (thalach)
  * ST depression induced by exercise (oldpeak)
  * Number of major vessels colored by fluoroscopy (ca)
  * Chest pain type (cp)

4. Clinical Implications:
- The models show strong potential for supporting medical diagnosis
- Decision Tree and Random Forest models are particularly promising for clinical use
- Feature importance analysis provides insights into key diagnostic indicators
- High accuracy across multiple models suggests reliable predictive capability

5. Limitations and Future Work:
- Consider collecting more data to improve model robustness
- Explore ensemble methods combining multiple models
- Validate findings with external datasets
- Investigate potential biases in the dataset
- Consider implementing real-time prediction system for clinical use

6. Model Selection Recommendations:
- For highest accuracy: Use the tuned Decision Tree model
- For balanced performance: Consider Logistic Regression
- For interpretability: Decision Tree provides clear decision rules
- For robust predictions: Random Forest offers good generalization

This analysis demonstrates the potential of machine learning in supporting heart disease diagnosis while highlighting the importance of model selection and feature engineering. The high performance across multiple models suggests strong predictive capability for heart disease diagnosis.